In [1]:
import requests
import pandas as pd

base_url = "https://api.themoviedb.org/3/movie/popular"
base_url_genres = "https://api.themoviedb.org/3/genre/movie/list"

with open("tmdbapikey.txt", "r") as file:
    api_key = file.read().strip()

all_movies = []

for page in range(1, 51):
    params = {
        "api_key": api_key,
        "language": "en-US",
        "page": page
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()

    if 'results' in data:
        movies = data['results']
    else:
        movies = []

    for movie in movies:
        info = {
            "id": movie.get("id"),
            "title": movie.get("title"),
            "original_title": movie.get("original_title"),
            "release_date": movie.get("release_date"),
            "overview": movie.get("overview"),
            "genre_ids": movie.get("genre_ids"),
            "original_language": movie.get("original_language"),
            "popularity": movie.get("popularity"),
            "vote_average": movie.get("vote_average"),
            "vote_count": movie.get("vote_count"),
            "adult": movie.get("adult"),
        }
        all_movies.append(info)

movie_df = pd.DataFrame(all_movies)

# turn genre ids into names instead of numbers
params_genres = {
    "api_key": api_key,
    "language": "en-US"
}
genre_response = requests.get(base_url_genres, params=params_genres)

genres_data = genre_response.json()

genre_mapping = {genre['id']: genre['name'] for genre in genres_data['genres']}

movie_df['genre_names'] = movie_df['genre_ids'].apply(lambda ids: [genre_mapping.get(genre_id) for genre_id in ids])

unique_genres = set()
for genres in movie_df['genre_names']:
    unique_genres.update(genres)

print("Unique Genres:", unique_genres)

movie_df.to_csv("my_api_data.csv", index=False)

len(unique_genres)

Unique Genres: {'Action', 'Science Fiction', 'Adventure', 'Thriller', 'History', 'Documentary', 'Crime', 'Horror', 'Fantasy', 'Romance', 'Comedy', 'Drama', 'Western', 'War', 'Mystery', 'Family', 'TV Movie', 'Animation', 'Music'}


19